In [1]:
pip install jupyter transformers faiss-cpu datasets


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install torch torchvision torchaudio jupyter transformers faiss-cpu datasets


Note: you may need to restart the kernel to use updated packages.


In [3]:
import torch
print(torch.__version__)


2.2.1


In [4]:
pip install --upgrade transformers datasets


Note: you may need to restart the kernel to use updated packages.


In [5]:
from transformers import RagTokenizer, RagTokenForGeneration, RagRetriever
import torch
from datasets import load_dataset


In [6]:
# Load the SQuAD dataset from Hugging Face datasets
dataset = load_dataset("squad")
print("Sample data:", dataset['train'][0])


Sample data: {'id': '5733be284776f41900661182', 'title': 'University_of_Notre_Dame', 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.', 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?', 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}


In [7]:
# Initialize the tokenizer and the RAG model
tokenizer = RagTokenizer.from_pretrained("facebook/rag-token-nq")
model = RagTokenForGeneration.from_pretrained("facebook/rag-token-nq")


C:\Users\siddh\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
C:\Users\siddh\anaconda3\Lib\site-packages\transformers\models\bart\configuration_bart.py:175: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected to

In [8]:
def ask_question(context, question):
    # Encode the inputs
    inputs = tokenizer(question, context, return_tensors='pt')
    
    # Generate an answer using the RAG model
    with torch.no_grad():
        answer_ids = model.generate(input_ids=inputs['input_ids'])
    
    # Decode and print the answer
    return tokenizer.decode(answer_ids[0], skip_special_tokens=True)

# Example usage
context = dataset['train'][0]['context']
question = dataset['train'][0]['question']
print("Context:", context)
print("Question:", question)
print("Answer:", ask_question(context, question))


Context: Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.
Question: To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?


AttributeError: 'NoneType' object has no attribute 'shape'

In [9]:
from transformers import RagTokenizer, RagTokenForGeneration, RagConfig, RagRetriever

# Load the RAG tokenizer and model with a retriever
tokenizer = RagTokenizer.from_pretrained("facebook/rag-token-base")
retriever = RagRetriever.from_pretrained("facebook/rag-token-base", dataset="wiki_dpr", index_name="compressed")

# Load the RAG model configured to use the retriever
config = RagConfig.from_pretrained("facebook/rag-token-base")
model = RagTokenForGeneration.from_pretrained("facebook/rag-token-base", config=config, retriever=retriever)


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called fr

ChunkedEncodingError: ('Connection broken: IncompleteRead(58495994 bytes read, 486734477 more expected)', IncompleteRead(58495994 bytes read, 486734477 more expected))

In [ ]:
def ask_question(question):
    # Encode the input question
    inputs = tokenizer(question, return_tensors="pt")

    # Generate an answer using the RAG model
    with torch.no_grad():
        outputs = model.generate(input_ids=inputs['input_ids'])

    # Decode the generated ids to get the answer
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

# Example question
question = "What is the capital of France?"
print("Answer:", ask_question(question))


In [ ]:
from transformers import BertTokenizer, BertForQuestionAnswering
import torch

# Load the tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

def answer_question(question, context):
    # Tokenize input question and context
    inputs = tokenizer.encode_plus(question, context, add_special_tokens=True, return_tensors="pt")
    input_ids = inputs["input_ids"].tolist()[0]

    # Model inference
    outputs = model(**inputs)
    answer_start_scores, answer_end_scores = outputs.start_logits, outputs.end_logits

    # Find the tokens with the highest `start` and `end` scores
    answer_start = torch.argmax(answer_start_scores)
    answer_end = torch.argmax(answer_end_scores) + 1

    # Convert tokens to answer string
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))
    
    return answer

# Example usage
context = dataset['train'][0]['context']
question = dataset['train'][0]['question']
print("Context:", context)
print("Question:", question)
print("Answer:", answer_question(question, context))


In [ ]:
for i in range(10):
    context = dataset['train'][i]['context']
    question = dataset['train'][i]['question']
    print("Context:", context)
    print("Question:", question)
    print("Answer:", answer_question(question, context))